In this notebook we explore the `evaluate` function offered by `ranx`.

First of all we need to install [ranx](https://github.com/AmenRa/ranx)

Mind that the first time you run any ranx' functions they may take a while as they must be compiled first

In [2]:
!pip install -U ranx

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.4 MB/s eta 0:00:00
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=54c860549b1a3f01c998a1f3b1f1588a9d407f41d10db6d2946acea1f5ed6f69
  Stored in directory: /root/.cache/pip/wheels/63/f9/dc/2dd16d3330e327236e4d407941975c42d5159d200cdb7922d8
  Created wheel for cbor: filename=cbor-1.0.0-cp311-cp311-linux_x86_64.whl size=5393

Download the data we need

In [3]:
import os
import requests

for file in ["qrels", "results"]:
    os.makedirs("notebooks/data", exist_ok=True)

    with open(f"notebooks/data/{file}.test", "w") as f:
        master = f"https://raw.githubusercontent.com/AmenRa/ranx/master/notebooks/data/{file}.test"
        f.write(requests.get(master).text)

In [4]:
from ranx import Qrels, Run, evaluate

In [5]:
qrels = Qrels.from_file("notebooks/data/qrels.test", kind="trec")
run = Run.from_file("notebooks/data/results.test", kind="trec")

Evaluate

For a full list of the available metrics see [here](https://amenra.github.io/ranx/metrics/).

In [ ]:
# Single metric
print(evaluate(qrels, run, "hits"))
print(evaluate(qrels, run, "hit_rate"))
print(evaluate(qrels, run, "precision"))
print(evaluate(qrels, run, "recall"))
print(evaluate(qrels, run, "f1"))
print(evaluate(qrels, run, "r-precision"))
print(evaluate(qrels, run, "mrr"))
print(evaluate(qrels, run, "map"))
print(evaluate(qrels, run, "ndcg"))
print(evaluate(qrels, run, "bpref"))
print(evaluate(qrels, run, "rbp.95"))

In [6]:
# Single metric with cutoff
evaluate(qrels, run, "ndcg@10")

/usr/local/lib/python3.11/dist-packages/ranx/metrics/ndcg.py:72: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _ndcg(qrels[i], run[i], k, rel_lvl, jarvelin)


np.float64(0.30157719921022785)

In [ ]:
# Multiple metrics
evaluate(qrels, run, ["map", "mrr", "ndcg"])

In [ ]:
# Multiple metrics with cutoffs (you can use different cutoffs for each metric)
evaluate(qrels, run, ["map@100", "mrr@10", "ndcg@10"])

In [ ]:
# By default, scores are saved in the evaluated Run
# You can disable this behaviour by passing `save_results_in_run=False`
# when calling `evaluate`
run.mean_scores

In [ ]:
import json  # Just for pretty printing

print(json.dumps(run.scores, indent=4))

# 301, 302, and 303 are the query ids

In [7]:
# Alternatively, per query scores can be extracted as Numpy Arrays by passing
# `return_mean = False` to `evaluate`
print(qrels)
evaluate(qrels, run, ["map@100", "mrr@10", "ndcg@10"], return_mean=False)

{'map@100': array([0.01179319, 0.39827964, 0.0764098 ]),
 'mrr@10': array([0.16666667, 1.        , 0.        ]),
 'ndcg@10': array([0.15176219, 0.75296941, 0.        ])}

In [ ]:
# Finally, you can set the number of threads used for computing the metric
# scores, by passing `threads = n` to `evaluate`
# `threads = 0` by default, which means all the available threads will be used
# Note that if the number of queries is small, `ranx` will automatically set
# `threads = 1` to prevent performance degradations
evaluate(qrels, run, ["map@100", "mrr@10", "ndcg@10"], threads=1)